In [5]:
import torchtext
from torchtext.vocab import Vectors
import torch
import numpy as np
import random


In [14]:
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 100
MAX_VOCAB_SIZE = 50000


TEXT = torchtext.legacy.data.Field(lower=True)
train, val, test = torchtext.legacy.datasets.LanguageModelingDataset.splits(path='.', train='text.8.train.txt', validation='text8.dev.txt', test='text8.test.txt', text_field=TEXT)
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)

TEXT.vocab.itos
# TEXT.vocab.stoi
device = torch.device("cuda" if USE_CUDA else "cpu")
train_iter, val_iter, test_iter = torchtext.legacy.data.BPTTIterator.splits((train, val, text),batch_szie = BATCH_SIZE, device=device,
                                                                            bptt_len = 50, repeat = FLASE, shuffle = True)

it = iter(train_iter)
batch=next(it)
" ".join(TEXT.vocab.itos[i] for i in batch.text[:, 0].data) # source text

" ".join(TEXT.vocab.itos[i] for i in batch.target[:, 0].data) # target text
len(TEXT.vocab)


FileNotFoundError: [Errno 2] No such file or directory: './text.8.train.txt'

In [15]:
import torch.nn as nn
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(RNNModule, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.hidden_size = hidden_size
        
    def forward(self, text, hidden):        
        #text: seq_length * batch_size * embed_size
        emb = self.embed(text)
        output = self.lstm(emb, hidden)
        # output: seq_size * batch_size * hidden_size
        # hidden: (1 * batch_size * hidden_size, 1 * batch_size * hidden_size)
        # output = output.view(-1, output.shape[2]) # (seq_length * batch_size) * hidden_size
        out_vocab = self.linear(output.view(-1, output.shape[2]))
        out_vocab = out_vocab.view(output.size(0), output.size(1), out_vocab(2))
        return out_vocab, hidden
    
    def init_hidden(self, bsz, requires_grad=True):
        weight = next(self.parameters())
        return (weight.new_zeros((1, bsz, self.hidden_size), requires_grad=True),
                weight.new_zeros((1, bsz, self.hidden_size), requires_grad=True))

In [1]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

model = RNNModel(vocab_size = len(TEXT.vocab), embed_size=EMBEDDING_SIZE, 
                hidden_size=HIDDEN_SIZE)


VOCAB_SIZE = len(TEXT.vocab)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Admin(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)
NUM_EPOCHS = 2
GRAD_CLIP = 5.

def evaluate(model, data):
    model.eval()
    total_loss = 0
    total_count = 0.
    it = iter(data)
    with torch.no_grad():
        hidden = model.init_hidden(BATCH_SIZE, requires_grad=False)
        for i, batch in enumerate(it):
            data, target = batch.text, batch.target
            hidden = repackage_hidden(hidden)
            output, hidden = model(data, hidden) # backpropgate through all 
            loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1)) # batch_size * target_class_dim, batch_size
            total_loss = loss.item() * np.muliply(*data.size())
            total_count = np.muliply(*data.size())
    loss = total_loss/ total_count
    model.tain()
    return loss
    
val_losses = []
for epoch in range(NUM_EPOCHS):
    model.train()
    it = iter(train_iter)
    hidden = model.init_hidden(BATCH_SIZE)
    
    for i, batch in enumerate(it):
        data, target = batch.text, batch.target
        hidden = repackage_hidden(hidden)
        output, hidden = model(data, hidden) # backpropgate through all 
        loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1)) # batch_size * target_class_dim, batch_size
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.paramters(), GRAD_CLIP)
        optimizer.step()
        if i % 100 = 0:
            print('loss', loss.item())

        if i % 10000 == 0:
            val_loss = evaluate(model, val_iter)
            if len(val_losses) == 0 or val_loss < min(val_losses):                
                torch.save(model.state_dict(), 'lp.pth')
            else:
                # learning rate decay
                scheduler.step()
            val_losses.append(val_loss)
                
            

SyntaxError: invalid syntax (<ipython-input-1-f999f738907e>, line 24)

In [2]:
best_model = RNNModel(vocab_size = len(TEXT.vocab), embed_size=EMBEDDING_SIZE, 
                hidden_size=HIDDEN_SIZE)
best_model.load_state_dict(torch.load('lm.pth'))
hidden = best_mode.init_hidden(1)
device = torch.device("cpu")
input = torch.randint(VOCOB_SIZE, (1, 1), dtype=torch.long).to(device)
words = []
for i in range(100):
    output, hidden = best_model(input, hidden)
    word_weights = output.squeeze().exp().cpu() # 增加分布之间的差值， logtis exp
    word_idx = torch.multinomial(word_weights, 1)[0] # multinomial sampling, greedy, argmax, 需要得到确定结果的话，使用argmax
    input.fill_(word_idx) 
    word = TEXT.vocab.itos[word_idx]
    words.append(word)
    
print(' '.join(words))

NameError: name 'RNNModel' is not defined